<a href="https://colab.research.google.com/github/shruti-sriniv/PCOS-Data-Modelling/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Importing the necessary modules in python
import numpy as np
import pandas as pd
import os

In [4]:
symptoms_df = pd.read_csv('PCOS_extended_dataset.csv')
print(symptoms_df.head(10))

   Sl. No  Patient File No.  PCOS (Y/N)   Age (yrs)  Weight (Kg)  Height(Cm)   \
0     193               193           0          30    69.979147   167.708055   
1     360               360           0          36    63.711688   154.055877   
2      10                10           0          36    51.848631   149.059804   
3     278               278           1          29    66.893988   148.628036   
4      71                71           0          33    52.536198   150.767409   
5     473               473           1          26    67.408647   165.951097   
6     397               397           0          31    68.703324   159.593469   
7     315               315           1          34    52.659470   157.495627   
8     487               487           1          30    49.764163   158.961647   
9      88                88           0          40    57.576129   156.596777   

         BMI  Blood Group  Pulse rate(bpm)   RR (breaths/min)  ...  \
0  23.185569           12             

In [5]:
# Adding in more import statements needed for visualization and model building
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [6]:
symptoms_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 44 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Sl. No                  2000 non-null   int64  
 1   Patient File No.        2000 non-null   int64  
 2   PCOS (Y/N)              2000 non-null   int64  
 3    Age (yrs)              2000 non-null   int64  
 4   Weight (Kg)             2000 non-null   float64
 5   Height(Cm)              2000 non-null   float64
 6   BMI                     2000 non-null   float64
 7   Blood Group             2000 non-null   int64  
 8   Pulse rate(bpm)         2000 non-null   int64  
 9   RR (breaths/min)        2000 non-null   int64  
 10  Hb(g/dl)                2000 non-null   float64
 11  Cycle(R/I)              2000 non-null   int64  
 12  Cycle length(days)      2000 non-null   int64  
 13  Marraige Status (Yrs)   1997 non-null   float64
 14  Pregnant(Y/N)           2000 non-null   